# SVM via Tensorflow

## 1. Theory

Given $(X_i, y_i), y_i=+1/-1,$ we try to maximize 
$$ \gamma = \frac{|W^T \cdot X + b|}{||W||}$$
With restriction
$$ 
\begin{cases}
    W^T \cdot X_i + b \geq +1, & \text{if } y_i=1\\ 
    W^T \cdot X_i + b \leq -1, & \text{if } y_i=-1
\end{cases}
$$

### 1.1 Equation Formalization
Using mathmetic refactoration, we get our **SVM algorithm goal formalized** as follow:
$$
\begin{equation}
\max_{W,b} \frac{2}{||W||}, \\
\text{ s.t. } y_i(W^T \cdot X_i + b) \geq 1, i = 1,2,...,m
\end{equation}
$$

Futhermore, $||w|| > 0$, we can equally use:
$$
\begin{equation}
\min_{W,b} \frac{||W||^2}{2}, \\
\text{ s.t. } y_i(W^T \cdot X_i + b) \geq 1, i = 1,2,...,m
\end{equation}
$$


### 1.2 Dual Problem
